<div style="  padding: 10px;text-align: center;" class='row'>
<div style="float:left;width: 10%;" class='column'><a href="https://datos.gov.co/"><img alt="Logo DataSandbox"  src="https://github.com/DataSandbox/Plantilla-Publicacion-Resultados/raw/main/App/logdat.JPG" style="width: 100px;"></a></div>
    <div style="float:left;width: 80%;" class='column'>
        <h1>EXTRACCIÓN DE INFORMACIÓN EN DOCUMENTOS DE PROCESOS DE CONTRATACIÓN PÚBLICA
        </h1> 
    </div>
 <div style="float:left;width: 10%;" class='column'><a href="https://www.colombiacompra.gov.co/" target="_blank"><img class="float-right" src="https://raw.githubusercontent.com/ANCP-CCE-Analitica/datasandbox-extraccion/main/logo_ancp_cce_web.png" style="width: 200px;"></a></div>
    </div>


# Uso de Text Analytics

Posterior a la creación del corpus, una lectura exitosa de 10368 contratos, procedemos a extraer la entidades con Text Analytics. El resultado es una base con el UID del proceso, el texto extraido, el texto limpio y las entidades reconcidas:

In [ ]:
import pandas as pd
spark.conf.set (
  "fs.azure.account.key.dlscontratacionpuproceso.dfs.core.windows.net",
  dbutils.secrets.get (scope = "contratacionpuproceso-key-vault-secrets", key = "StorageAccountKey")
)
file_location = "abfss://demo2@dlscontratacionpuproceso.dfs.core.windows.net/Descarga SECOP I/"



In [ ]:
DFText_total=spark.read.parquet(file_location+'/Descarga SECOP I/MUESTRA/DFText_total.parquet')

In [ ]:
DFText_total=DFText_total.toPandas()

In [ ]:
DFText_total_processed=DFText_total[['UID','Text']]

Antes, una estandarización del texto muy sencilla:

In [ ]:
New_corpus=[]
for i in DFText_total_processed.Text:
      New_corpus.append(i.replace('\n',' ').replace('\t',' ').replace('-','').upper())
DFText_total_processed['clean_text']=New_corpus
  
                                            

## Implementación de Text Analytics
Creamos una base por proceso que determina las entidades sus categorías y subcategorías y nos indica donde está cada entidad encontrada:

In [ ]:
!pip install azure.ai.textanalytics

In [ ]:
key = "08cc21d9ec66400cb32024f4dce8842f"
endpoint = "https://cog-text-contratacionpublica-procesos.cognitiveservices.azure.com/"

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using your key and endpoint 
def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

# Example function for recognizing entities from text


In [ ]:
def entity_recognition_example(client):

    try:
        documents = ["En el monte de la china una china se perdió con 500 millones de pesos y la policía no la encontró..."]
        result = client.recognize_entities(documents = documents)[0]

        print("Named Entities:\n")
        for entity in result.entities:
            print("\tText: \t", entity.text, "\tCategory: \t", entity.category, "\tSubCategory: \t", entity.subcategory,
                    "\n\tConfidence Score: \t", round(entity.confidence_score, 2), "\tLength: \t", entity.length, "\tOffset: \t", entity.offset, "\n")

    except Exception as err:
        print("Encountered exception. {}".format(err))
entity_recognition_example(client)

In [ ]:
import numpy as np

entities=dict()
for num,i in enumerate(DFText_total_processed['clean_text']):
  num_max=int(np.ceil(len(i)/5120))
  documents=[]
  if num%100==0:
    print(num)
  for j in range(num_max):
    documents.append(i[j*5120:(j+1)*5120])
  num_max_2=int(np.ceil(len(documents)/5))
  documents_2=[]
  for j in range(num_max_2):
    documents_2.append(documents[j*5:(j+1)*5])
  other_list=dict()
  Texto=[]
  Categoría=[]
  Subcategoría=[]
  Confianza=[]
  Longitud=[]
  Compensación=[]
  for h in documents_2:
    for k in client.recognize_entities(documents = h):
      for entity in k.entities:
        Texto.append(entity.text)
        Categoría.append(entity.category)
        Subcategoría.append(entity.subcategory)
        Confianza.append(round(entity.confidence_score, 2))
        Compensación.append(entity.offset)
        Longitud.append(entity.length)
      other_list['Texto']=Texto
      other_list['Categoría']=Categoría
      other_list['Subcategoría']=Subcategoría
      other_list['Confianza']=Confianza
      other_list['Longitud']=Longitud
      other_list['Compensación']=Compensación    
  entities[num]=other_list

Una visualización de lo obtenido en el contrato 10.000:

In [ ]:
pd.DataFrame(entities[10000]).head(50)

In [ ]:
DFText_total_processed['Entidades']=entities.values()

In [ ]:
DFText_total_processedspark=spark.createDataFrame(DFText_total_processed)
DFText_total_processedspark.write.parquet(file_location+'/Resultado/DFText_total_entities_total.parquet',mode='overwrite')